# Multiple Linear Regression

In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

import seaborn as sns

import statsmodels.api as sm
from statsmodels.graphics.gofplots import ProbPlot
from statsmodels.stats.outliers_influence import variance_inflation_factor

def fit_linear_reg(x, y):
    '''Fit Linear model with predictors x on y 
    return AIC, BIC, R2 and R2 adjusted '''
    x = sm.add_constant(x)
    # Create and fit model
    model_k = sm.OLS(y, x).fit()
    
    # Find scores
    BIC = model_k.bic
    AIC = model_k.aic
    R2 = model_k.rsquared
    R2_adj = model_k.rsquared_adj
    RSS = model_k.ssr
    
    # Return result in Series
    results = pd.Series(data={'BIC': BIC, 'AIC': AIC, 'R2': R2,
                              'R2_adj': R2_adj, 'RSS': RSS})
    
    return results


def add_one(x_full, x, y, scoreby='RSS'):
    ''' Add possible predictors from x_full to x, 
    Fit a linear model on y using fit_linear_reg
    Returns Dataframe showing scores as well as best model '''
    # Predefine DataFrame
    x_labels = x_full.columns
    zeros = np.zeros(len(x_labels))
    results = pd.DataFrame(
        data={'Predictor': x_labels.values, 'BIC': zeros, 
               'AIC': zeros, 'R2': zeros, 
               'R2_adj': zeros, 'RSS': zeros})

    # For every predictor find R^2, RSS, and AIC
    for i in range(len(x_labels)):
        x_i = np.concatenate((x, [np.array(x_full[x_labels[i]])]))
        results.iloc[i, 1:] = fit_linear_reg(x_i.T, y)
        
    # Depending on where we scoreby, we select the highest or lowest
    if scoreby in ['RSS', 'AIC', 'BIC']:
        best = x_labels[results[scoreby].argmin()]
    elif scoreby in ['R2', 'R2_adj']:
        best = x_labels[results[scoreby].argmax()]
        
    return results, best 

def drop_one(x, y, scoreby='RSS'):
    ''' Remove possible predictors from x, 
    Fit a linear model on y using fit_linear_reg
    Returns Dataframe showing scores as well as predictor 
    to drop in order to keep the best model '''
    # Predefine DataFrame
    x_labels = x.columns
    zeros = np.zeros(len(x_labels))
    results = pd.DataFrame(
        data={'Predictor': x_labels.values, 'BIC': zeros, 
               'AIC': zeros, 'R2': zeros, 
               'R2_adj': zeros, 'RSS': zeros})

    # For every predictor find RSS and R^2
    for i in range(len(x_labels)):
        x_i = x.drop(columns=x_labels[i])
        results.iloc[i, 1:] = fit_linear_reg(x_i, y)
    
    # Depending on where we scoreby, we select the highest or lowest
    if scoreby in ['RSS', 'AIC', 'BIC']:
        worst = x_labels[results[scoreby].argmin()]
    elif scoreby in ['R2', 'R2_adj']:
        worst = x_labels[results[scoreby].argmax()]
    
    return results, worst 


""" Plot Residuals vs Fitted Values """
def plot_residuals(axes, res, yfit, n_samp=0):
    """ Inputs:
    axes: axes created with matplotlib.pyplot
    x: x values
    ytrue: y values
    yfit: fitted/predicted y values
    res[optional]: Residuals, used for resampling
    n_samp[optional]: number of resamples """
    # For every random resampling
    for i in range(n_samp):
        # 1. resample indices from Residuals
        samp_res_id = random.sample(list(res), len(res))
        # 2. Average of Residuals, smoothed using LOWESS
        sns.regplot(x=yfit, y=samp_res_id,
        scatter=False, ci=False, lowess=True,
        line_kws={'color': 'lightgrey', 'lw': 1, 'alpha': 0.8})
        # 3. Repeat again for n_samples

    dataframe = pd.concat([yfit, res], axis=1)
    axes = sns.residplot(x=yfit, y=res, data=dataframe, 
                         lowess=True, scatter_kws={'alpha': 0.5}, 
                         line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})
    axes.set_title('Residuals vs Fitted')
    axes.set_ylabel('Residuals')
    axes.set_xlabel('Fitted Values')
    
""" QQ Plot standardized residuals """
def plot_QQ(axes, res_standard, n_samp=0):
    """ Inputs:
    axes: axes created with matplotlib.pyplot
    res_standard: standardized residuals
    n_samp[optional]: number of resamples """
    
    # QQ plot instance
    QQ = ProbPlot(res_standard)
    # Split the QQ instance in the seperate data
    qqx = pd.Series(sorted(QQ.theoretical_quantiles), name="x")
    qqy = pd.Series(QQ.sorted_data, name="y")
    if n_samp != 0:
        # Estimate the mean and standard deviation
        mu = np.mean(qqy)
        sigma = np.std(qqy)
        # For ever random resampling
        for lp in range(n_samp):
            # Resample indices
            samp_res_id = np.random.normal(mu, sigma, len(qqx))
            # Plot
            sns.regplot(x=qqx, y=sorted(samp_res_id),
            scatter=False, ci=False, lowess=True,
            line_kws={'color': 'lightgrey', 'lw': 1, 'alpha': 0.8})

            sns.regplot(x=qqx, y=qqy, scatter=True, lowess=False, 
                        ci=False, scatter_kws={'s': 40, 'alpha': 0.5}, 
                        line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})
            
    axes.plot(qqx, qqx, '--k', alpha=0.5)
    axes.set_title('Normal Q-Q')
    axes.set_xlabel('Theoretical Quantiles')
    axes.set_ylabel('Standardized Residuals')

""" Scale-Location Plot """
def plot_scale_loc(axes, yfit, res_stand_sqrt, n_samp=0):
    """ Inputs:
    axes: axes created with matplotlib.pyplot
    yfit: fitted/predicted y values
    res_stand_sqrt: Absolute square root Residuals
    n_samp[optional]: number of resamples """
    
    # For every random resampling
    for i in range(n_samp):
    # 1. resample indices from sqrt Residuals
        samp_res_id = random.sample(list(res_stand_sqrt), len(res_stand_sqrt))
        # 2. Average of Residuals, smoothed using LOWESS
        sns.regplot(x=yfit, y=samp_res_id,
                    scatter=False, ci=False, lowess=True,
                    line_kws={'color': 'lightgrey', 'lw': 1, 'alpha': 0.8})
        # 3. Repeat again for n_samples

    # plot Regression usung Seaborn
    sns.regplot(x=yfit, y=res_stand_sqrt,
                scatter=True, ci=False, lowess=True,
                scatter_kws={'s': 40, 'alpha': 0.5},
                
    line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})
    axes.set_title('Scale-Location plot')
    axes.set_xlabel('Fitted values')
    axes.set_ylabel('$\sqrt{\|Standardized\ Residuals\|}$')

def plot_cooks(axes, res_inf_leverage, res_standard, n_pred=1,
               x_lim=None, y_lim=None, n_levels=4):
    """ Inputs:
    axes: axes created with matplotlib.pyplot
    res_inf_leverage: Leverage
    res_standard: standardized residuals
    n_pred: number of predictor variables in x
    x_lim, y_lim[optional]: axis limits
    n_levels: number of levels"""
    
    sns.regplot(x=res_inf_leverage, y=res_standard,
                scatter=True, ci=False, lowess=True,
                scatter_kws={'s': 40, 'alpha': 0.5},
                
    line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})
    # Set limits
    if x_lim != None:
        x_min, x_max = x_lim[0], x_lim[1]
    else:
        x_min, x_max = min(res_inf_leverage)*0.95, max(res_inf_leverage)*1.05
    if y_lim != None:
        y_min, y_max = y_lim[0], y_lim[1]
    else:
        y_min, y_max = min(res_standard)*0.95, max(res_standard)*1.05

    # Plot centre line
    plt.plot((x_min, x_max), (0, 0), 'g--', alpha=0.8)
    # Plot contour lines for Cook's Distance levels
    n = 100
    cooks_distance = np.zeros((n, n))
    x_cooks = np.linspace(x_min, x_max, n)
    y_cooks = np.linspace(y_min, y_max, n)

    for xi in range(n):
        for yi in range(n):
            cooks_distance[yi][xi] = \
            y_cooks[yi]**2 * x_cooks[xi] / (1 - x_cooks[xi]) / (n_pred + 1)
            
    CS = axes.contour(x_cooks, y_cooks, cooks_distance, levels=n_levels, alpha=0.6)

    axes.clabel(CS, inline=0, fontsize=10)
    axes.set_xlim(x_min, x_max)
    axes.set_ylim(y_min, y_max)
    axes.set_title('Residuals vs Leverage and Cook\'s distance')
    axes.set_xlabel('Leverage')
    axes.set_ylabel('Standardized Residuals')
    
""" Standard scatter plot and regression line """
def plot_reg(axes, x, y, x_lab="x", y_lab="y", title="Linear Regression"):
    """ Inputs:
    axes: axes created with matplotlib.pyplot
    x: (single) Feature
    y: Result """
    # Plot scatter data
    sns.regplot(x=x, y=y,
                scatter=True, ci=False, lowess=False,
                scatter_kws={'s': 40, 'alpha': 0.5},
                
    line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})
    # Set labels:
    axes.set_xlabel(x_lab)
    axes.set_ylabel(y_lab)
    axes.set_title(title)

""" VIF Analysis """
def VIF_analysis(x):
    """ VIF analysis of variables saved in x
    Input:
    x: m*n matrix or Dataframe, containing m samples of n predictors
    Output:
    VIF: Vector containing n Variance Inflation Factors
    """
    # Preproces:
    x_np = x.to_numpy()
    VIF = []
    # For all n Predictors:
    for i in range(x.shape[1]):
        # Define x and y
        x_i = np.delete(x_np, i, 1)
        x_i = sm.add_constant(x_i)
        y_i = x_np[:, i]
        # Fit model
        model = sm.OLS(y_i, x_i).fit()
        # Calculate the VIF
        VIF.append(1 / (1 - model.rsquared))
    
    return VIF

def plot_resid_analysis(model):
    # Find the predicted values for the original design.
    yfit = model.fittedvalues
    # Find the Residuals
    res = model.resid
    # Influence of the Residuals
    res_inf = model.get_influence()
    # Studentized residuals using variance from OLS
    res_standard = res_inf.resid_studentized_internal
    # Absolute square root Residuals:
    res_stand_sqrt = np.sqrt(np.abs(res_standard))
    # Cook's Distance and leverage:
    res_inf_cooks = res_inf.cooks_distance
    res_inf_leverage = res_inf.hat_matrix_diag

    """ Plots """
    # Create Figure and subplots
    fig = plt.figure(figsize = (12,9))

    # First subplot: Residuals vs Fitted values with 100 resamples
    ax1 = fig.add_subplot(2, 2, 1)
    plot_residuals(ax1, res, yfit, n_samp = 100)

    # Second subplot: QQ Plot with 100 resamples
    ax2 = fig.add_subplot(2, 2, 2)
    plot_QQ(ax2, res_standard, n_samp = 100)

    # Third subplot: Scale-location with 100 resamples
    ax3 = fig.add_subplot(2, 2, 3)
    plot_scale_loc(ax3, yfit, res_stand_sqrt, n_samp = 100)

    # Fourth subplot: Residuals vs Fitted values with 100 resamples
    ax4 = fig.add_subplot(2, 2, 4)
    plot_cooks(ax4, res_inf_leverage, res_standard, n_pred = model.df_model)

    return


#### Daten laden und Modell fitten

In [ ]:
import statsmodels.api as sm
import pandas as pd
import matplotlib.pyplot as plt

# Load data
df = pd.read_csv('./data/fitness.csv')

# Prepare Data
x = df[['age','weight','runtime','rstpulse','runpulse','maxpulse']] # Predictor Variables
x_sm = sm.add_constant(x)       # Add constant 
y = df['oxy']                   # Response Variable

model = sm.OLS(y, x_sm).fit()   # Fit the model
print(model.summary())          # Print summary 

#### Residuenanalyse (zuerst helper script laden!)

In [ ]:
plot_resid_analysis(model)

#### Korrelationsmatrix

In [ ]:
# Response Variable 'oxy' wird entfernt
print(round(df.drop(['oxy'],axis=1).corr(), 4))

#### VIF

In [ ]:
# (1 = GUT, ab 5-10 hinweis für Multikolinearität)
VIF = VIF_analysis(x)

res = pd.DataFrame(data={'Predictor': x.columns,
                         'VIF': np.round(VIF, 3)})
print(res.T)

#### VIF FIX

In [ ]:
# new variable durch kombinationen von Variablen mit hohem VIF
x_ii = df.drop(['oxy', 'maxpulse'], axis=1) # Zielevariable 'oxy' und 'maxpulse' (hohes VIF) entfernen
newvar = df['runpulse'] / df['maxpulse'] # neue Variable erstellen 
newvar = newvar.rename('run/max')
x_ii = x_ii.join(newvar) # neue Variable hinzufügen

#### Paarweise Vergleich (zweier Modelle)

In [ ]:
pred_i = model_i.predict(sm.add_constant(x_i))
pred_ii = model_ii.predict(sm.add_constant(x_ii))

fig = plt.figure(figsize = (6,5))
ax1 = fig.add_subplot(1, 1, 1)

plot_reg(ax1, pred_i, pred_ii,
         x_lab="pred_i", y_lab="pred_ii",
         title="Comparing predictions")
plt.show()

# Liegen die Punkte auf der Diagonalen, sind die Vorhersagen 
# identisch, falls nicht ist durch die exlusion von variablen 
# precision verloren gegangen

#### Forward Stepwise Model Selection

In [ ]:
results = pd.DataFrame(data={'Best_Pred': [], 'AIC':[]})

# Define the empty predictor
x0 = [np.zeros(len(y))]

x_red = x.copy() # Copy the full set of predictors
x_forward = x0


for i in range(x_red.shape[1]):
    results_i, best_i = add_one(x_red, x_forward, y, scoreby='AIC')

    # Update the empty predictor with the best predictor
    x_forward = np.concatenate((x_forward, [x[best_i]]))

    # Remove the chosen predictor from the list of options
    x_red = x_red.drop(columns=best_i)

    # Save results
    results.loc[i, 'Best_Pred'] = best_i
    results.loc[i, 'AIC'] = results_i['AIC'].min()
    results.loc[i, 'R2'] = results_i['R2'].min()
    results.loc[i, 'RSS'] = results_i['RSS'].min()

print('Best Predictors and corresponding AIC:\n', results,
'\n\nThe best model thus contains',
results['AIC'].argmin() + 1, ' predictors') # change this to needed metric

#### Backward Stepwise Model Selection

#### Anova

In [ ]:
# 4.1 f) anova test
x_opt = x_ii[['runtime', 'run/max', 'age', 'weight']]
x_opt_sm = sm.add_constant(x_opt)  # Add constant
model_opt = sm.OLS(y, x_opt_sm).fit()  # Fit the model

# Anova Test:
table = sm.stats.anova_lm(model, model_opt)
print('\nANOVA Test:\n', table)